In [51]:
import pandas as pd
import datetime

investment_dict = [{'401(k) Plan Balance':'ubiquity'},
                  {'Health Savings Account':'fidelity_hsa'},
                  {'Individual - TOD':'fidelity_tod'},
                  {'Individual Brokerage':'etrade_tod'},
                  {'Roth IRA':'etrade_roth_ira'}]

credit_card_list = ['American Express Gold Card','Delta SkyMiles Gold Card',
                    'Delta SkyMiles Gold Card - Supplementary account','Additional Gold Card - Supplementary account',
                    'Credit Card','CREDITCARD Account']


In [62]:
##Dataset is from 13 Jan 2020. Creating the dataframe in this step.
df1=pd.read_csv("mint_transactions_latest.csv")
expense_category_df=pd.read_csv("expense_category.csv")

##Rename few columns
df1.rename(columns={'Transaction Type':'Transaction_Type',
                  'Account Name':'Account_Name'}, inplace=True)

##get the account categorization
accounts = pd.read_csv("accounts.txt")

##Joining the transactions with account categorization table
df = pd.merge(df1,accounts,on='Account_Name')


##Creating few time dimensions
df['Date'] = pd.to_datetime(df['Date'],format='%m/%d/%Y')
df['year_month'] = df['Date'].dt.to_period('M')
df['month'] = df['Date'].dt.month
df['week'] = df['Date'].dt.isocalendar().week


print("Number of transaction -" ,len(df))

Number of transaction - 7160


In [53]:
#Fixing certain wrong categorization done by mint based on exploratory analysis
df.loc[df['Description']=='STEAMSHIP AUTHORITY WOODS HOLE MA', ['Category']]='Ferry fare'
df.loc[df['Description']=='RELIANCE RETAIL LTD Mumbai MA',['Category']]='Clothing'
df.loc[df['Description']=='1947 NORWOOD 0000 NORWOOD MA',['Category']]='Restaurants'
df.loc[df['Description'].str.contains("Sid"),['Category']]='phone bill'
df.loc[df['Description']=='South Shore Desi',['Category']]  ='Groceries'
df.loc[df['Description'].str.contains('AMZ_STORECRD_PMT'),['Category']]='Amazon Shopping'
df.loc[df['Description'].str.contains('MARATHON SPORTS'),['Category']]='fitness'


In [63]:
##Question 1 - How much I have recived in dividend across all accounts
dividend_df = df[df['Category']=='Dividend & Cap Gains'].sort_values(by='Amount', ascending=False)
dividend_df[['Amount','Account_Name']].groupby('Account_Name').sum('Amount').reset_index().sort_values(by='Amount', ascending=False)

,Account_Name,Amount
0,401(k) Plan Balance,918.47
2,Individual - TOD,693.86
4,Roth IRA,368.62
1,Health Savings Account,22.80
3,Individual Brokerage,0.39


In [64]:
#Question 2 - How much I have paid in interest to credit card companies
credit_card_df=df[df['Account_Name'].isin (credit_card_list)]
credit_card_df['Category'].unique()
credit_card_interest = credit_card_df[credit_card_df['Category'].isin (['Finance Charge'])][['Amount','Transaction_Type']]
credit_card_interest['absolute_amount'] = credit_card_interest.apply(lambda x :(-1)*x[0] if x[1]=='debit' else x[0], axis=1)
print('Interest paid on credit card dues:',credit_card_interest['absolute_amount'].sum())

Interest paid on credit card dues: -179.31


In [56]:
#Question 3 - How much I spent on food every month
df['Category'].unique()
food_catg_list = list(expense_category_df[expense_category_df['category_level2']=='Food']['category_level1'])
print(food_catg_list)
food_catg_df = df[df['Category'].isin(food_catg_list)]
food_catg_expense_df = food_catg_df[['Category','year_month','Amount']].groupby(['year_month','Category']).sum('Amount').reset_index()
food_expense_monthly = food_catg_expense_df[['year_month','Amount']].groupby(['year_month']).sum('Amount').reset_index()
food_expense_monthly.tail(6)

['Groceries', 'Fast Food', 'Restaurants', 'Coffee Shops', 'Food & Dining', 'Food Delivery']


,year_month,Amount
25,2022-02,679.87
26,2022-03,689.82
27,2022-04,988.51
28,2022-05,1229.66
29,2022-06,1450.92
30,2022-07,108.49


In [65]:
##Resturant expense every month
restra_df=food_catg_expense_df[(food_catg_expense_df['Category']=='Restaurants')]
restra_df.tail(6)

,year_month,Category,Amount
96,2022-02,Restaurants,228.88
100,2022-03,Restaurants,119.09
105,2022-04,Restaurants,377.77
111,2022-05,Restaurants,418.10
117,2022-06,Restaurants,595.80
120,2022-07,Restaurants,90.39


In [66]:
##Groceries expense every month
grocery_df=food_catg_expense_df[(food_catg_expense_df['Category']=='Groceries')]
grocery_df.tail()

,year_month,Category,Amount
99,2022-03,Groceries,512.95
104,2022-04,Groceries,456.68
110,2022-05,Groceries,590.31
116,2022-06,Groceries,637.52
119,2022-07,Groceries,12.75


In [68]:
##calculating credit & debit for the latest month(June 2022)
earn_year_month='2022-05'
spend_year_month='2022-06'

##All the money came in account
credit_df=df[(df['year_month']==earn_year_month) & \
             (df['Account_Name']=='Adv Plus Banking - 9491') & \
             (df['Transaction_Type']=='credit') ]

##All the spend happened across all credit card and bank account - Excluding the credit card payments
debit_df=df[(df['year_month']==spend_year_month) & \
            (df['Transaction_Type']=='debit') & \
            (df['Account_Name'].isin(['American Express Gold Card', \
                                      'Adv Plus Banking - 9491',' \
                                      Amazon Prime Store Card', \
                                      'Discover', \
                                      'Additional Gold Card - Supplementary account', \
                                      'Delta SkyMiles Gold Card', \
                                       'Delta SkyMiles Gold Card - Supplementary account',\
                                      'CREDITCARD Account'
                                     ]) &
            (~df['Category'].isin(['Credit Card Payment'])))
           ]


#### Report from the last month
#Total earnings
print("Total Income:-" , credit_df['Amount'].sum())
#Total Spend
print("Total Spend:-" , debit_df['Amount'].sum())


##spend across all the credit card
print("Spend Per Account:-")
debit_df_grp=debit_df[['Account_Name','Amount']].groupby('Account_Name').sum('Amount').reset_index()
print(debit_df_grp.sort_values(by='Amount',ascending=False).to_string(index=False))
print()

#Money left
print("Money saved:-" , round(credit_df['Amount'].sum() - debit_df['Amount'].sum(),2))
print()





Total Income:- 6539.2
Total Spend:- 7490.66
Spend Per Account:-
                                    Account_Name  Amount
                         Adv Plus Banking - 9491 2995.00
                              CREDITCARD Account 1971.85
                      American Express Gold Card 1860.67
                        Delta SkyMiles Gold Card  477.20
    Additional Gold Card - Supplementary account  131.39
Delta SkyMiles Gold Card - Supplementary account   54.55

Money saved:- -951.46



In [60]:
## All expenditure across all the credit cards across different category 
agg_debit_df= debit_df[['Amount','Category']].groupby('Category').sum('Amount').reset_index().sort_values(by='Amount', ascending=False)
print("spend across all category")
print(agg_debit_df.to_string(index=False))

spend across all category
              Category  Amount
       Mortgage & Rent 1808.38
             Groceries  637.52
  Babysitter & Daycare  599.00
           Restaurants  595.80
                  Hair  530.81
            Gas & Fuel  377.11
           Investments  295.00
                   Gym  279.00
              Transfer  265.00
                 Hotel  257.09
       Amazon Shopping  200.73
                Doctor  160.00
               fitness  140.00
            Cash & ATM  140.00
              Pharmacy  128.12
              Clothing  124.89
           Service Fee  123.26
             Fast Food  114.52
            phone bill   77.33
            Television   69.99
          Coffee Shops   60.54
             Utilities   52.74
      Auto & Transport   50.00
              Shopping   48.06
        Finance Charge   42.14
         Entertainment   40.00
         Movies & DVDs   38.16
     Business Services   32.61
         Food Delivery   31.54
                 Books   27.60
            H